In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os


In [2]:
df = pd.read_csv(os.path.join("..", "Resources", "beer.csv"))
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,type,address,website,state,state_breweries
0,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
1,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
2,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
3,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
4,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361241 entries, 0 to 361240
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   brewery_id          361241 non-null  int64  
 1   brewery_name        361241 non-null  object 
 2   review_time         361241 non-null  int64  
 3   review_overall      361241 non-null  float64
 4   review_aroma        361241 non-null  float64
 5   review_appearance   361241 non-null  float64
 6   review_profilename  361241 non-null  object 
 7   beer_style          361241 non-null  object 
 8   review_palate       361241 non-null  float64
 9   review_taste        361241 non-null  float64
 10  beer_name           361241 non-null  object 
 11  beer_abv            361241 non-null  float64
 12  beer_beerid         361241 non-null  int64  
 13  type                361241 non-null  object 
 14  address             361241 non-null  object 
 15  website             361241 non-nul

In [4]:
target = df["review_overall"]

target.value_counts()





4.0    142517
4.5     82365
3.5     66898
3.0     31370
5.0     20775
2.5      9364
2.0      5445
1.5      1434
1.0      1073
Name: review_overall, dtype: int64

In [6]:
medianreview=df.loc[:,["review_aroma","review_appearance","review_palate","review_taste"]].median()

In [8]:
medianreview

review_aroma         4.0
review_appearance    4.0
review_palate        4.0
review_taste         4.0
dtype: float64

In [5]:
df.iloc[:,4].value_counts()

4.0    141229
3.5     85863
4.5     66263
3.0     38878
5.0     13232
2.5      9610
2.0      4877
1.5       850
1.0       439
Name: review_aroma, dtype: int64

In [ ]:
target.replace([3.5,3],"C",inplace=True)
target.replace([4.5,5],"A",inplace=True)
target.replace([2.5,2,1.5,1],"D",inplace=True)
target.replace([4],"B",inplace=True)

In [ ]:
unused = df.drop(["review_overall","website","address","brewery_name", "brewery_id", "review_time","review_profilename"], axis=1)
unused.head()